In [189]:
import pandas as pd
import numpy as np
import random
import datetime

### dimFactories

In [190]:
factories = pd.DataFrame(pd.read_csv('./done_csv/dimOrder.csv')['factory_code'].unique()).rename({0: 'factory_code'}, axis=1)

In [191]:
factories['factory_id'] = factories.index + 1
factories['factory_name'] = pd.Series([ "Vinnytsia", "Buffalo", "Bila Tserkva", "Deva", "Weimar", "Znojmo",
    "Royan", "Montevideo", "Clifton", "Daugavpils", "Skadovsk", "Vajjak", "Norwich", "Tesinets", "Espoo",
    "Arpad", "Thessaloniki", "Blankenburg", "Dnipro", "Utsen", "Skovorodynivka", "Dagestan", "Savastopol"
])

In [192]:
factories = factories.iloc[:, [1, 0, 2]]

In [193]:
factories.to_csv('./done_csv/dimFactory.csv', index=None)

In [194]:
factories.to_json('./done_json/dimFactory.json', orient='records')

### dimParcel

In [195]:
orders = pd.read_csv('./done_csv/dimOrder.csv')
products = pd.read_csv('./done_csv/dimProduct.csv')

In [196]:
orders['order_date'] = orders['order_date'].apply(pd.to_datetime).dt.date

C:\Users\Ivan\AppData\Local\Temp\ipykernel_10616\1397814682.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  orders['order_date'] = orders['order_date'].apply(pd.to_datetime).dt.date


In [197]:
order_sample = orders.sort_values('order_date', ascending=False)
order_sample['product_name'] = order_sample['product_name'].str.strip()

In [198]:
def get_qt_per_parcil(product_name):
    product = products.loc[products['product_name'] == product_name]
    if product.size != 0:
        return product.iloc[0].loc['qt_per_parcel']
    else:
        return None

In [199]:
order_sample['qt_per_parcel'] = order_sample['product_name'].apply(get_qt_per_parcil)

In [200]:
order_sample['order_qt'] = order_sample['order_qt'].astype(float)
order_sample['qt_per_parcel'] = order_sample['qt_per_parcel'].astype(float)

In [201]:
order_sample['parcels_qt_full'] = pd.Series(np.ceil(order_sample['order_qt'] / order_sample['qt_per_parcel']))

In [202]:
order_sample = order_sample.loc[~order_sample['parcels_qt_full'].isna()].head(300)

In [203]:
def get_order_details(order):
    return order_sample.loc[order_sample['order_code'] == order].iloc[0]

def get_product_details(product_name):
    return products.loc[products['product_name'] == product_name].iloc[0]

In [204]:
parcels = pd.DataFrame(columns=['parcel_id', 'fk_order_id','№ in order', 'parcel_status', 'parcel_created_date', 'qt_in_parcel', 'coil_name', 'parcel_name'])

In [205]:
counter = 0
for order_code in order_sample['order_code']:
    n_parcels = int(order_sample.loc[order_sample['order_code'] == order_code]['parcels_qt_full'].values[0])
    order_date = order_sample.loc[order_sample['order_code'] == order_code]['order_date'].values[0]
    last_date = order_date
    end_qt, single_qt = get_order_details(order_code).loc[['order_qt', 'qt_per_parcel']]
    last_qt = end_qt % single_qt
    for n in range(n_parcels):
        quantity = [single_qt, last_qt][n == n_parcels - 1]
        parcels.loc[counter]= [counter+1, order_code, n+1, 0, last_date, quantity, "", ""]
        last_date += datetime.timedelta(days=random.choices([0, 1, 2, 3], [.5, .3, .15, .05])[0])
        counter += 1

In [206]:
parcels

,parcel_id,fk_order_id,№ in order,parcel_status,parcel_created_date,qt_in_parcel,coil_name,parcel_name
0,1,BIL248211499,1,0,2024-12-07,1600.0,,
1,2,BIL248211499,2,0,2024-12-07,1600.0,,
2,3,BIL248211499,3,0,2024-12-07,1600.0,,
3,4,BIL248211499,4,0,2024-12-08,1600.0,,
4,5,BIL248211499,5,0,2024-12-09,1600.0,,
...,...,...,...,...,...,...,...,...
4630,4631,VIL240621447,14,0,2024-06-21,1500.0,,
4631,4632,VIL240621447,15,0,2024-06-24,1500.0,,
4632,4633,VIL240621447,16,0,2024-06-25,1500.0,,
4633,4634,VIL240621447,17,0,2024-06-25,1500.0,,


In [207]:
parcels.to_csv('./done_csv/dimParcels.csv', index=None)

In [208]:
parcels.to_json('./done_json/dimParcels.json', orient='records')

### factParcelRun

In [209]:
parcel_runs = pd.DataFrame(columns=['fk_parcel_id', 'varnish_type', 'run_number', 'varnish_amount_kg', 'solvent_amount_kg', 'run_status', 'run_date'])

In [210]:
def get_parcel_details(parcel_id):
    return parcels.loc[parcels['parcel_id'] == parcel_id].iloc[0]

def get_run_data(parcel_id):
    return parcel_runs.loc[parcel_runs['fk_parcel_id'] == parcel_id]

def get_varnishings(parcel_id):
    return get_product_details(get_order_details(get_parcel_details(parcel_id).loc['fk_order_id']).loc['product_name']).loc[['varnishing_1', 'varnishing_2', 'varnishing_3', 'varnishing_4']]

In [211]:
counter = 0
for parcel in parcels['parcel_id']:
    varnishings = get_varnishings(int(parcel)).dropna()
    parcel_date = get_parcel_details(parcel).loc['parcel_created_date']
    last_date = parcel_date
    for i in range(len(varnishings)):
        parcel_runs.loc[counter] = [parcel, varnishings[i], i+1, random.randint(10, 1500), random.randint(1, 500), 1, last_date]
        last_date += datetime.timedelta(days=random.choices([1, 2, 3, 4], [.5, .3, .15, .05])[0])
        counter += 1

C:\Users\Ivan\AppData\Local\Temp\ipykernel_10616\401199584.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  parcel_runs.loc[counter] = [parcel, varnishings[i], i+1, random.randint(10, 1500), random.randint(1, 500), 1, last_date]
C:\Users\Ivan\AppData\Local\Temp\ipykernel_10616\401199584.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  parcel_runs.loc[counter] = [parcel, varnishings[i], i+1, random.randint(10, 1500), random.randint(1, 500), 1, last_date]
C:\Users\Ivan\AppData\Local\Temp\ipykernel_10616\401199584.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version

In [212]:
parcel_runs

,fk_parcel_id,varnish_type,run_number,varnish_amount_kg,solvent_amount_kg,run_status,run_date
0,17,PL1019-34,1,819,439,1,2024-12-01
1,17,816361,2,324,39,1,2024-12-03
2,18,PL1019-34,1,464,377,1,2024-12-01
3,18,816361,2,730,22,1,2024-12-02
4,19,816314,1,593,468,1,2024-12-01
...,...,...,...,...,...,...,...
7614,4633,DE4488-3LV,2,656,380,1,2024-06-28
7615,4634,PL1019-34,1,166,28,1,2024-06-25
7616,4634,DE4488-3LV,2,24,400,1,2024-06-27
7617,4635,PL1019-34,1,306,32,1,2024-06-25


In [213]:
get_run_data(104)

,fk_parcel_id,varnish_type,run_number,varnish_amount_kg,solvent_amount_kg,run_status,run_date
131,104,PL1019-34,1,897,244,1,2024-11-08
132,104,DE4488-3LV,2,522,421,1,2024-11-11
133,104,DE5449-4,3,51,57,1,2024-11-14


In [214]:
parcel_runs.to_csv('./done_csv/factParcelRuns.csv', index=None)

In [215]:
parcel_runs.to_json('./done_json/factParcelRuns.json', orient='records')